In [ ]:
import asyncio
import logging
import sys

from astropy.coordinates import Angle
from astropy.time import Time

from lsst.ts import salobj
from lsst.ts.observatory.control.maintel.mtcs import MTCS
from lsst.ts.observatory.control.maintel.comcam import ComCam

In [ ]:
stream_handler = logging.StreamHandler(sys.stdout)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
domain = salobj.Domain()
#mtcs = MTCS(domain)
comcam = ComCam(domain)
mtptg = salobj.Remote(domain, "MTPtg")

#await asyncio.gather(mtcs.start_task, comcam.start_task)
await asyncio.gather(comcam.start_task, mtptg.start_task)

In [ ]:
mtptg.cmd_azElTarget.set(
    targetName="Park position",
    azDegs=Angle(0.0, unit=u.deg).deg,
    elDegs=Angle(80.0, unit=u.deg).deg,
    rotPA=Angle(0.0, unit=u.deg).deg
)

In [ ]:
ack = await mtptg.cmd_azElTarget.start(timeout=30)
print(ack)

In [ ]:
await comcam.standby()